In [4]:
import numpy as np
import pandas as pd
from copy import deepcopy
#from google.colab import drive

from scipy.spatial.distance import cosine
from scipy.spatial.distance import jaccard

In [5]:
#drive.mount('/content/drive')

# Read the dataset

In [23]:
data = pd.read_csv("kmeans_data\data.csv", header=None)
label = pd.read_csv("kmeans_data\label.csv", header=None)

# convert into np array
data = data.values
label = label.values

# Fix a seed

In [24]:
seed_value = 7

# Helper functions

In [8]:
def distance(metric_name, point_a, point_b):
  if metric_name == "euclidean":
    return euclidean_distance(point_a, point_b)
  elif metric_name == "cosine":
    return cosine_distance(point_a, point_b)
  elif metric_name == "jaccard":
    return jaccard_distance(point_a, point_b)

  return point_a - point_b

def euclidean_distance(point_a, point_b):
  return np.linalg.norm(point_a - point_b)

def cosine_distance(point_a, point_b):
  return cosine(point_a, point_b)

def jaccard_distance(point_a, point_b):
  return jaccard(point_a, point_b)

# Implementation of Kmeans

In [25]:
def k_means(k, distance_metrics_name, stopping_criteria, max_iteration=200, threshold=0.1):
  # select k data points randomly as the centroids
  np.random.seed(seed_value)

  random_indices = np.random.choice(data.shape[0], k, replace=False)
  centroids = data[random_indices]

  iteration = 0
  temporary_clusters = [-1] * len(data)
  distances_from_centroid = [np.inf] * len(data)

  sse = np.inf

  while True:
    iteration += 1

    # for all the data points find out the nearest centroid
    for i in range(len(data)):
      distances_from_centroid[i] = np.inf

      for j in range(k):
        d = distance(distance_metrics_name, centroids[j], data[i])
        if d < distances_from_centroid[i]:
          distances_from_centroid[i] = d
          temporary_clusters[i] = j

    current_sse = sum(distances_from_centroid)

    # find out new centroids
    new_centroids = []

    for i in range(k):
      # the data points that belongs to the ith centroid
      matching_indices = [j for j, value in enumerate(temporary_clusters) if value == i]

      subset_of_data = data[matching_indices]
      mean = np.mean(subset_of_data, axis=0)

      new_centroids.append(mean)

    # stopping criteria
    if stopping_criteria == "max_iteration" and iteration >= max_iteration:
      break
    elif stopping_criteria == "sse_increase" and sse < current_sse:
      break
    else:
      # check whether the centroids changed
      flag = True
      for i in range(k):
          diff = np.abs(centroids[i] - new_centroids[i])
          if np.any(diff) > threshold:
              flag = False

      if flag:
          break

    sse = current_sse
    centroids = deepcopy(new_centroids)

    # worst case scenario
    if iteration > 700:
      break

  print("Total iteration required:", iteration)
  print("SSE:", sse)

  # Accuracy
  clusters = [{} for i in range(k)]
  for i in range(len(data)):
    if temporary_clusters[i] != -1:
      actual_label = label[i][0]

      if actual_label not in clusters[temporary_clusters[i]]:
        clusters[temporary_clusters[i]][actual_label] = 0

      clusters[temporary_clusters[i]][actual_label] += 1

  accuracies = 0
  for i in range(k):
    try:
      mx = max(clusters[i].values())
      sm = sum(clusters[i].values())

      accuracies += mx / sm
    except:
      accuracies += 0

  print("Accuracy:", accuracies / k * 100)

In [26]:
unique_elements, counts = np.unique(label, return_counts=True)
number_of_clusters = len(unique_elements)
print("Number of clusters:", number_of_clusters)

Number of clusters: 10


## Distance metric: Euclidean

In [27]:
print("Distance metric: Euclidean. Stopping criteria: unchanged centroid")
k_means(number_of_clusters, "euclidean", stopping_criteria="centroid_change")
print()

print("Distance metric: Euclidean. Stopping criteria: increased SSE")
k_means(number_of_clusters, "euclidean", stopping_criteria="sse_increase")
print()

print("Distance metric: Euclidean. Stopping criteria: max iteration of 150")
k_means(number_of_clusters, "euclidean", stopping_criteria="max_iteration", max_iteration=150)
print()

Distance metric: Euclidean. Stopping criteria: unchanged centroid
Total iteration required: 51
SSE: 15664678.69503718
Accuracy: 62.926861884213366

Distance metric: Euclidean. Stopping criteria: increased SSE
Total iteration required: 51
SSE: 15664678.69503718
Accuracy: 62.926861884213366

Distance metric: Euclidean. Stopping criteria: max iteration of 150
Total iteration required: 51
SSE: 15664678.69503718
Accuracy: 62.926861884213366



## Distance metric: Cosine

In [12]:
print("Distance metric: Cosine. Stopping criteria: unchanged centroid")
k_means(number_of_clusters, "cosine", stopping_criteria="centroid_change")
print()

print("Distance metric: Cosine. Stopping criteria: increased SSE")
k_means(number_of_clusters, "cosine", stopping_criteria="sse_increase")
print()

print("Distance metric: Cosine. Stopping criteria: max iteration of 150")
k_means(number_of_clusters, "cosine", stopping_criteria="max_iteration", max_iteration=500)
print()

Distance metric: Cosine. Stopping criteria: unchanged centroid
Total iteration required: 112
SSE: 2466.808536926088
Accuracy: 66.46915110377626

Distance metric: Cosine. Stopping criteria: increased SSE
Total iteration required: 110
SSE: 2466.803476088712
Accuracy: 66.4759390798922

Distance metric: Cosine. Stopping criteria: max iteration of 150
Total iteration required: 112
SSE: 2466.808536926088
Accuracy: 66.46915110377626



In [13]:
print("Distance metric: Jaccard. Stopping criteria: unchanged centroid")
k_means(number_of_clusters, "jaccard", stopping_criteria="centroid_change")
print()

print("Distance metric: Jaccard. Stopping criteria: increased SSE")
k_means(number_of_clusters, "jaccard", stopping_criteria="sse_increase")
print()

print("Distance metric: Jaccard. Stopping criteria: max iteration of 150")
k_means(number_of_clusters, "jaccard", stopping_criteria="max_iteration", max_iteration=500)
print()

Distance metric: Jaccard. Stopping criteria: unchanged centroid


C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Total iteration required: 701
SSE: 10000.0
Accuracy: 1.135

Distance metric: Jaccard. Stopping criteria: increased SSE
Total iteration required: 2
SSE: 9489.942228051723
Accuracy: 9.309989946043

Distance metric: Jaccard. Stopping criteria: max iteration of 150
Total iteration required: 150
SSE: 10000.0
Accuracy: 1.135

